In [1]:
import itk
from itk import TubeTK as tube

import numpy as np

In [2]:
InputBaseName = "../data/CT/CT"
OutputBaseName = "results/CT-Lungs"

InputFilename = OutputBaseName+".mha"

ImageType = itk.Image[itk.F, 3]
im1Iso = itk.imread(OutputBaseName+"-VesselsEnhanced.mha")
im1Enh = itk.imread(OutputBaseName+"-VesselsEnhanced-Masked.mha")

spacing = im1Iso.GetSpacing()[0]

In [3]:
imMath = tube.ImageMath.New(im1Enh)
imMath.MedianFilter(1)
imMath.Threshold(0.000001, 1, 1, 0)
im1VessMask = imMath.GetOutputShort()

ccSeg = tube.SegmentConnectedComponents.New(im1VessMask)
ccSeg.SetMinimumVolume(100)
ccSeg.Update()
im1VessMask = ccSeg.GetOutput()

itk.imwrite(im1VessMask,OutputBaseName+"-VesselsInitialMask.mha")
im1VessMask = itk.imread(OutputBaseName+"-VesselsInitialMask.mha", itk.F)

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


In [4]:
imMath.SetInput(im1Enh)
imMath.ReplaceValuesOutsideMaskRange(im1VessMask,1,99999,0)
im1Seeds = imMath.GetOutput()
itk.imwrite(im1Seeds,OutputBaseName+"-VesselSeeds.mha")

In [5]:
imMath.SetInput(im1VessMask)
imMath.Threshold(0,0,1,0)
im1VessMaskInv = imMath.GetOutput()

distFilter = itk.DanielssonDistanceMapImageFilter.New(im1VessMaskInv)
distFilter.Update()
dist = distFilter.GetOutput()

imMath.SetInput(dist)
imMath.Blur(0.4)
tmp = imMath.GetOutput()
imMath.ReplaceValuesOutsideMaskRange(tmp, 0.1, 10, 0)
im1SeedsRadius = imMath.GetOutput()

itk.imwrite(im1SeedsRadius, OutputBaseName+"-VesselSeedsRadius.mha")

In [6]:
imMath.SetInput(im1Iso)
imMath.ReplaceValuesOutsideMaskRange(im1VessMask, 1, 99999, 0)
imMath.Blur(spacing)
im1Input = imMath.GetOutput()

itk.imwrite(im1Input,OutputBaseName+"-VesselInput.mha")

In [7]:
numSeeds = 300

vSeg = tube.SegmentTubes.New(Input=im1Input)
#vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0.0001)
vSeg.SetMinRoundness(0.1)
vSeg.SetMinRidgeness(0.75)
vSeg.SetMinLevelness(0.001)
vSeg.SetRadiusInObjectSpace( spacing )
vSeg.SetBorderInIndexSpace(3)
vSeg.SetSeedMask( im1Seeds )
vSeg.SetSeedRadiusMask( im1SeedsRadius )
vSeg.SetOptimizeRadius(True)
vSeg.SetSeedMaskMaximumNumberOfPoints(numSeeds)
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedExtractionMinimumProbability(0.5)
vSeg.ProcessSeeds()

In [8]:
tubeMaskImage = vSeg.GetTubeMaskImage()
itk.imwrite(tubeMaskImage,OutputBaseName+"-Vessels.mha")

In [9]:
SOWriter = itk.SpatialObjectWriter[3].New()
SOWriter.SetInput(vSeg.GetTubeGroup())
SOWriter.SetBinaryPoints(True)
SOWriter.SetFileName(OutputBaseName+"-Vessels.tre")
SOWriter.Update()

In [10]:
# smooth tubes!
TubeMath = tube.TubeMath[3, itk.F].New()
TubeMath.SetInputTubeGroup(vSeg.GetTubeGroup())
TubeMath.SetUseAllTubes()
TubeMath.SmoothTube(4,"SMOOTH_TUBE_USING_INDEX_GAUSSIAN")
TubeMath.SmoothTubeProperty("Radius",2,"SMOOTH_TUBE_USING_INDEX_GAUSSIAN")
tubes = TubeMath.GetOutputTubeGroup()

In [11]:
ConvSurface = tube.WriteTubesAsPolyData.New()
ConvSurface.SetInput(tubes)
ConvSurface.SetFileName(OutputBaseName+"-Vessels.vtp")
ConvSurface.Update()